In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pandas as pd
import warnings
import matplotlib.colors as colors
import matplotlib.animation as animation
import matplotlib.lines as lines
import datetime
import math
import csv
import ast
%matplotlib notebook

In [2]:
warnings.filterwarnings(action='once')

!pip install sociophysicsdatahandler
from sociophysicsDataHandler import SociophysicsDataHandler

!cat auth.txt

dh = SociophysicsDataHandler()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\utils\_process_win32.py:145: ResourceWarning: unclosed file <_io.BufferedWriter name=4>
  return process_handler(cmd, _system_body)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\utils\_process_win32.py:145: ResourceWarning: unclosed file <_io.BufferedReader name=5>
  return process_handler(cmd, _system_body)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\utils\_process_win32.py:145: ResourceWarning: unclosed file <_io.BufferedReader name=6>
  return process_handler(cmd, _system_body)


guest
YDPGL-XREXC-QGMZS-UAKES


C:\ProgramData\Anaconda3\lib\site-packages\IPython\utils\_process_win32.py:145: ResourceWarning: unclosed file <_io.BufferedWriter name=4>
  return process_handler(cmd, _system_body)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\utils\_process_win32.py:145: ResourceWarning: unclosed file <_io.BufferedReader name=5>
  return process_handler(cmd, _system_body)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\utils\_process_win32.py:145: ResourceWarning: unclosed file <_io.BufferedReader name=6>
  return process_handler(cmd, _system_body)


In [3]:



def fetchData(path):
  #Fetch and combine all data from a directory
  file_list = dh.list_files(path)
  print(dh.filelist)
  n = file_list.name.size
  print(dh.filelist.head(10))
  df_array = np.empty(n, dtype=object)

  for i in range(n):
    dh.fetch_prorail_data_from_path(path + "/" + file_list.name[i]);
    #Convert the date time column to actual date time
    dh.df['date_time_utc'] = pd.to_datetime(dh.df['date_time_utc'], unit = 'ms')
    #Swap x and y
    column_titles = {'date_time_utc':'date_time_utc', 'tracked_object':'tracked_object', 'x_pos':'y_pos', 'y_pos':'x_pos'}
    dh.df.rename(columns = column_titles, inplace = True)

    df_array[i] = dh.df

  return pd.concat(df_array)


file_path = 'ehv/platform2.1/20220509/'


# fetch the data from the first file
df = fetchData(file_path);



dh.fetch_background_image_from_path('ehv/background_images/EHV.Perron2.1_multisensor.png')



targeting path /storage/surfsara/ProRail_USE_LL_data/ehv/platform2.1/20220509/


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Files listed. Accessible as <this-object>.filelist
                                                 path  \
0   /storage/surfsara/ProRail_USE_LL_data/ehv/plat...   
1   /storage/surfsara/ProRail_USE_LL_data/ehv/plat...   
2   /storage/surfsara/ProRail_USE_LL_data/ehv/plat...   
3   /storage/surfsara/ProRail_USE_LL_data/ehv/plat...   
4   /storage/surfsara/ProRail_USE_LL_data/ehv/plat...   
5   /storage/surfsara/ProRail_USE_LL_data/ehv/plat...   
6   /storage/surfsara/ProRail_USE_LL_data/ehv/plat...   
7   /storage/surfsara/ProRail_USE_LL_data/ehv/plat...   
8   /storage/surfsara/ProRail_USE_LL_data/ehv/plat...   
9   /storage/surfsara/ProRail_USE_LL_data/ehv/plat...   
10  /storage/surfsara/ProRail_USE_LL_data/ehv/plat...   
11  /storage/surfsara/ProRail_USE_LL_data/ehv/plat...   
12  /storage/surfsara/ProRail_USE_LL_data/ehv/plat...   
13  /storage/surfsara/ProRail_USE_LL_data/ehv/plat...   
14  /storage/surfsara/ProRail_USE_LL_data/ehv/plat...   
15  /storage/surfsara/ProRail_USE_LL_

data fetched. Accessible as <this-object>.df
trying to fetch: /storage/surfsara/ProRail_USE_LL_data/ehv/platform2.1/20220509//EHV_Platform2.1_2022050913_trajectorie.parquet
data fetched. Accessible as <this-object>.df
trying to fetch: /storage/surfsara/ProRail_USE_LL_data/ehv/platform2.1/20220509//EHV_Platform2.1_2022050914_trajectorie.parquet
data fetched. Accessible as <this-object>.df
trying to fetch: /storage/surfsara/ProRail_USE_LL_data/ehv/platform2.1/20220509//EHV_Platform2.1_2022050915_trajectorie.parquet
data fetched. Accessible as <this-object>.df
trying to fetch: /storage/surfsara/ProRail_USE_LL_data/ehv/platform2.1/20220509//EHV_Platform2.1_2022050916_trajectorie.parquet
data fetched. Accessible as <this-object>.df
trying to fetch: /storage/surfsara/ProRail_USE_LL_data/ehv/platform2.1/20220509//EHV_Platform2.1_2022050917_trajectorie.parquet
data fetched. Accessible as <this-object>.df
trying to fetch: /storage/surfsara/ProRail_USE_LL_data/ehv/platform2.1/20220509//EHV_Platf

In [4]:


def above_line(df, line, alpha, x_tag='x_pos',y_tag='y_pos'):
    above_line_series = df[y_tag] > - math.atan(alpha/360*2*math.pi)*(df[x_tag]-line[0])+line[1]
    return above_line_series

def below_line(df, line, alpha, x_tag='x_pos',y_tag='y_pos'):
    below_line_series = df[y_tag] < - math.atan(alpha/360*2*math.pi)*(df[x_tag]-line[0])+line[1]
    return below_line_series

def left_of_line(df, line, alpha, x_tag='x_pos',y_tag='y_pos'):
    left_of_line_series = df[x_tag] < line[0]
    return left_of_line_series

df_grouped_by_id = df.groupby('tracked_object')
df_object_start_location = df_grouped_by_id.first()
df_object_end_location = df_grouped_by_id.last()

alpha = 0.8 
boarding_line_1 = [-2e4,2e3] #x, y, angle (degrees)
dx = 5e4
boarding_line_1.append(boarding_line_1[0]+dx) # [2]: x of point 2
boarding_line_1.append(boarding_line_1[1]-dx*math.tan(alpha/360*2*math.pi)) # [3]: y of point 2

boarding_line_2 = [-2e4,-0.65e4] #x, y, angle (degrees)
dx = 5e4
boarding_line_2.append(boarding_line_2[0]+dx) # [2]: x of point 2
boarding_line_2.append(boarding_line_2[1]-dx*math.tan(alpha/360*2*math.pi)) # [3]: y of point 2

boarding_line_3 = [70000,-15000,70000,10000]

boarding_line_4 = [10000,-15000,10000,10000]

#If start location is above line 1, offboarding. same for below line 2. Also to the right of the vertical line

offboarding_1 = above_line(df_object_start_location,boarding_line_1,alpha)
offboarding_2 = below_line(df_object_start_location,boarding_line_2,alpha)
offboarding_3 = (df_object_start_location['x_pos']>boarding_line_3[0])
offboarding = offboarding_1 | offboarding_2 | offboarding_3

#If end location is above line 1, onboarding. same for below line 2.

onboarding_1 = above_line(df_object_end_location,boarding_line_1,alpha)
onboarding_2 = below_line(df_object_end_location,boarding_line_2,alpha)
onboarding_3 = (df_object_end_location['x_pos']>boarding_line_3[0])
entering = left_of_line(df_object_start_location,boarding_line_4,0)
onboarding_or_entering = onboarding_1 | onboarding_2 | onboarding_3 | entering
onboarding_or_entering.head()


list_offboarding = offboarding.loc[offboarding==True].index.values
print(list_offboarding)


incoming_people_offboarding = df_object_start_location.loc[offboarding==True]
print(incoming_people_offboarding)


incoming_people_onboarding = df_object_start_location.loc[onboarding_or_entering==True]
#print(incoming_people_onboarding)

incoming_people_entering = df_object_start_location.loc[entering==True]
print(incoming_people_entering)






C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[5436765 5436778 5436779 ... 5476473 5476475 5476478]
                         date_time_utc   y_pos    x_pos  \
tracked_object                                            
5436765        2022-05-08 22:00:00.400  3133.0  47762.0   
5436778        2022-05-08 22:01:06.000   204.0  72908.0   
5436779        2022-05-08 22:01:06.700  3482.0  15312.0   
5436780        2022-05-08 22:01:10.500  -613.0  70481.0   
5436781        2022-05-08 22:01:18.100 -8028.0  72807.0   
...                                ...     ...      ...   
5476471        2022-05-09 21:22:50.800  2892.0  56922.0   
5476472        2022-05-09 21:23:22.900  2744.0  36012.0   
5476473        2022-05-09 21:23:55.000  3582.0   7330.0   
5476475        2022-05-09 21:24:59.000  3495.0   7521.0   
5476478        2022-05-09 21:43:19.700  2207.0   4020.0   

                                       datetime  
tracked_object                                   
5436765        2022-05-09 00:00:00.400000+02:00  
5436778               2022-0

In [5]:
df.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,date_time_utc,tracked_object,y_pos,x_pos,datetime
0,2022-05-08 22:00:00,5436643,-6024.0,25580.0,2022-05-09 00:00:00+02:00
1,2022-05-08 22:00:00,5436660,-3259.0,62973.0,2022-05-09 00:00:00+02:00
2,2022-05-08 22:00:00,5436661,-4228.0,17801.0,2022-05-09 00:00:00+02:00
3,2022-05-08 22:00:00,5436662,-3320.0,17744.0,2022-05-09 00:00:00+02:00
4,2022-05-08 22:00:00,5436664,-5976.0,26130.0,2022-05-09 00:00:00+02:00


In [6]:
def get_timestamp(utc_time):
    return int(datetime.datetime.timestamp(utc_time))

incoming_people_entering['date_time_unix']=incoming_people_entering['date_time_utc'].map(lambda a: get_timestamp(a))
incoming_people_offboarding['date_time_unix']=incoming_people_offboarding['date_time_utc'].map(lambda a: get_timestamp(a))
print(incoming_people_offboarding['date_time_unix'])

dictionary_people_entering = {}
dictionary_people_offboarding = {}

def filter_on_timestamp(unix_timestamp,df):
    temp_list = []
    temp_df = df.loc[df['date_time_unix']==unix_timestamp]
    for index, row in temp_df.iterrows():
        temp_list.append([row['x_pos'],row['y_pos']])
    return(temp_list)

start_datetime = get_timestamp(df.iloc[0]['datetime'])
print(start_datetime)
                              
for i in range(start_datetime, start_datetime+24*3600):
    if i%1000==0:
        print(i)
    
    temp = filter_on_timestamp(i,incoming_people_entering)
    if temp != []:
        dictionary_people_entering[i] = temp
        
    temp = filter_on_timestamp(i,incoming_people_offboarding)
    if temp != []:
        dictionary_people_offboarding[i] = temp

print(dictionary_people_entering)
print(dictionary_people_offboarding)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-6-8eff9845b8a3>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  incoming_people_entering['date_time_unix']=incoming_people_entering['date_time_utc'].map(lambda a: get_timestamp(a))


tracked_object
5436765    1652040000
5436778    1652040066
5436779    1652040066
5436780    1652040070
5436781    1652040078
              ...    
5476471    1652124170
5476472    1652124202
5476473    1652124235
5476475    1652124299
5476478    1652125399
Name: date_time_unix, Length: 14760, dtype: int64
1652047200
1652048000
1652049000
1652050000
1652051000
1652052000
1652053000
1652054000
1652055000
1652056000
1652057000
1652058000
1652059000
1652060000
1652061000
1652062000
1652063000
1652064000
1652065000
1652066000
1652067000
1652068000
1652069000
1652070000
1652071000
1652072000
1652073000
1652074000
1652075000
1652076000
1652077000
1652078000
1652079000
1652080000
1652081000
1652082000
1652083000
1652084000
1652085000
1652086000
1652087000
1652088000
1652089000
1652090000
1652091000
1652092000
1652093000
1652094000
1652095000
1652096000
1652097000
1652098000
1652099000
1652100000
1652101000
1652102000
1652103000
1652104000
1652105000
1652106000
1652107000
1652108000
1652109000


In [7]:
x=4
y=6
if 3<x<5<y<7:
    print('s')

s


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
#convert coordinates to grid locations
dict_people_entering_zones = dictionary_people_entering.copy()
dict_people_offboarding_zones = dictionary_people_offboarding.copy()

patch_width = 1000
patch_height = 1000
size = (patch_width, patch_height)

total_patches = (67,14)

o_x = 5000
o_y = -10000
origin = (o_x,o_y)

for key in dictionary_people_entering.keys():
    zones = []
    
    for pos in dictionary_people_entering[key]:
        x_pos = pos[0]
        x_pos = max(min(x_pos,71999), 5001) # 5000 72000
        y_pos = pos[1] 
        y_pos = max(min(y_pos,3999), -9999)# -10000 4000
        i = np.floor((x_pos-origin[0])/size[0])
        j = np.floor((y_pos-origin[1])/size[1])

        total_x_patches = total_patches[0]
        total_y_patches = total_patches[1]

        zones.append((i*total_y_patches)+j)
        
    dict_people_entering_zones[key] = zones

    
for key in dictionary_people_offboarding.keys():
    zones = []
    
    for pos in dictionary_people_offboarding[key]:
        x_pos = pos[0]
        x_pos = max(min(x_pos,71999), 5001) # 5000 72000
        y_pos = pos[1] 
        y_pos = max(min(x_pos,3999), -9999)# -10000 4000
        i = np.floor((x_pos-origin[0])/size[0])
        j = np.floor((y_pos-origin[1])/size[1])

        total_x_patches = total_patches[0]
        total_y_patches = total_patches[1]

        zones.append((i*total_y_patches)+j)
        
    dict_people_offboarding_zones[key] = zones


In [9]:
dict_people_offboarding_zones

{1652053528: [13.0],
 1652053819: [13.0],
 1652053903: [13.0],
 1652054781: [13.0],
 1652054805: [13.0],
 1652054926: [937.0],
 1652054976: [13.0],
 1652055234: [13.0],
 1652056444: [937.0, 937.0],
 1652057013: [13.0],
 1652057014: [13.0],
 1652058012: [937.0],
 1652059603: [13.0],
 1652059752: [13.0],
 1652059985: [937.0],
 1652060009: [755.0],
 1652060046: [937.0],
 1652060060: [13.0],
 1652060075: [461.0],
 1652060085: [13.0],
 1652060087: [237.0],
 1652060104: [97.0],
 1652060137: [111.0],
 1652060146: [111.0],
 1652060569: [13.0],
 1652060575: [13.0],
 1652060586: [209.0],
 1652060589: [209.0],
 1652061220: [937.0],
 1652061223: [923.0],
 1652061349: [125.0, 13.0, 615.0, 909.0, 265.0],
 1652061350: [13.0],
 1652061351: [111.0, 251.0, 265.0],
 1652061353: [937.0, 937.0],
 1652061355: [937.0],
 1652061356: [475.0],
 1652061361: [937.0],
 1652061366: [615.0],
 1652061375: [615.0],
 1652061376: [559.0],
 1652061381: [447.0],
 1652061401: [937.0, 937.0],
 1652061423: [447.0],
 16520622

In [12]:
with open('incoming_people_entering_9_5.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=';',escapechar=' ', quoting=csv.QUOTE_NONE)
    for key in dict_people_entering_zones.keys():
       csvwriter.writerow([key,dict_people_entering_zones[key]])
    
with open('incoming_people_offboarding_9_5.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=';',escapechar=' ', quoting=csv.QUOTE_NONE)
    for key in dict_people_offboarding_zones.keys():
       csvwriter.writerow([key,dict_people_offboarding_zones[key]])

In [13]:
dict_people_entering_zones

{1652053528: [3.0],
 1652053535: [3.0],
 1652053813: [4.0],
 1652053819: [3.0],
 1652053894: [5.0],
 1652053903: [3.0],
 1652054772: [4.0],
 1652054781: [2.0],
 1652054805: [2.0],
 1652054976: [2.0],
 1652055234: [3.0],
 1652055240: [3.0],
 1652055241: [3.0],
 1652055246: [3.0],
 1652057006: [4.0, 5.0],
 1652057013: [2.0],
 1652057014: [2.0],
 1652057916: [5.0],
 1652059588: [5.0],
 1652059603: [12.0],
 1652059738: [5.0],
 1652059752: [12.0],
 1652060060: [12.0],
 1652060061: [7.0],
 1652060085: [12.0],
 1652060388: [11.0],
 1652060458: [4.0],
 1652060490: [5.0],
 1652060503: [5.0],
 1652060533: [4.0],
 1652060569: [13.0],
 1652060575: [13.0],
 1652060827: [4.0],
 1652060837: [5.0],
 1652060901: [4.0],
 1652060905: [4.0],
 1652060948: [4.0],
 1652061132: [4.0],
 1652061349: [13.0],
 1652061350: [13.0],
 1652061364: [24.0],
 1652061440: [51.0],
 1652061442: [24.0],
 1652061559: [5.0],
 1652062035: [4.0],
 1652062280: [7.0],
 1652062326: [6.0],
 1652062355: [4.0],
 1652062484: [4.0],
 16